In [1]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data


data104925  data117808


In [ ]:

%cd ~/work/
!git clone https://hub.fastgit.org/PaddlePaddle/PaddleVideo.git
!ls /home/aistudio/work

In [2]:

!python3.7 -m pip install --upgrade pip
!python3.7 -m pip install --upgrade -r /home/aistudio/work/PaddleVideo/requirements.txt

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 1.7MB 12.2MB/s eta 0:00:01
  Found existing installation: pip 19.2.3
    Uninstalling pip-19.2.3:
      Successfully uninstalled pip-19.2.3
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 15.7 MB 52.0 MB/s            
     |████████████████████████████████| 11.3 MB 63.6 MB/s            
     |████████████████████████████████| 76 kB 16.9 MB/s            
     |████████████████████████████████| 262 kB 75.1 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 28.2 MB 59.2 MB/s            
     |████████████████████████████████| 11.8 MB 66.3 MB/s            
     |████████████████████████████████| 37.2 MB 70.0 MB/s            
  Created wheel for pyyaml: filename=PyYAML-4.2b2-cp37-cp37m-linux_x86_64.whl size=43031 sha256=8136413e8bd703fca22827f5bc16a06c10a4d3d406bd2266cebe9db148f5e255
  Stored 

In [18]:
%cd ~/work/PaddleVideo/

/home/aistudio/work/PaddleVideo


In [8]:
# 划分训练集和验证集，用验证集选择训练过程中在验证集上最优的模型，并用来对test data做预测
# 这里验证集的大小为150个样本
import numpy as np
original_data = np.load('/home/aistudio/data/data104925/train_data.npy')
original_label = np.load('/home/aistudio/data/data104925/train_label.npy')
permuted_idx = np.random.permutation(np.arange(len(original_label)))
train_idx = permuted_idx[150:]
train_data = original_data[train_idx]
train_label = original_label[train_idx]
valid_idx = permuted_idx[:150] 
valid_data = original_data[valid_idx]
valid_label = original_label[valid_idx]

# 训练集和验证集的路径
data_path = '/home/aistudio/split_dataset'
import os
if not os.path.isdir(data_path):
        os.makedirs(data_path)
np.save('/home/aistudio/split_dataset/train_data',train_data)
np.save('/home/aistudio/split_dataset/train_label',train_label)
np.save('/home/aistudio/split_dataset/valid_data',valid_data)
np.save('/home/aistudio/split_dataset/valid_label',valid_label)
print(train_data.shape,valid_data.shape)

(2772, 3, 2500, 25, 1) (150, 3, 2500, 25, 1)


In [1]:
# 给定的训练和测试的数据是关节点joint，这里产生骨骼bone
# reference:
# @inproceedings{2sagcn2019cvpr,  
#       title     = {Two-Stream Adaptive Graph Convolutional Networks for Skeleton-Based Action Recognition},  
#       author    = {Lei Shi and Yifan Zhang and Jian Cheng and Hanqing Lu},  
#       booktitle = {CVPR},  
#       year      = {2019},  
# }

# @article{shi_skeleton-based_2019,
#     title = {Skeleton-{Based} {Action} {Recognition} with {Multi}-{Stream} {Adaptive} {Graph} {Convolutional} {Networks}},
#     journal = {arXiv:1912.06971 [cs]},
#     author = {Shi, Lei and Zhang, Yifan and Cheng, Jian and LU, Hanqing},
#     month = dec,
#     year = {2019},
# }
# 具体参考的代码：https://hub.fastgit.org/lshiwjx/2s-AGCN/blob/master/data_gen/gen_bone_data.py

import numpy as np
from numpy.lib.format import open_memmap
from tqdm import tqdm

paris= [(1, 8), (0, 1), (15, 0), (17, 15), (16, 0),
                             (18, 16), (5, 1), (6, 5), (7, 6), (2, 1), (3, 2),
                             (4, 3), (9, 8), (10, 9), (11, 10), (24, 11),
                             (22, 11), (23, 22), (12, 8), (13, 12), (14, 13),
                             (21, 14), (19, 14), (20, 19)]
################train and valid############
datasets=('train','valid')
for dataset in datasets:
    data = np.load('split_dataset/{}_data.npy'.format(dataset))
    N, C, T, V, M = data.shape
    fp_sp = open_memmap(
        'split_dataset/{}_data_bone.npy'.format(dataset),
        dtype='float32',
        mode='w+',
        shape=(N, 3, T, V, M))

    fp_sp[:, :C, :, :, :] = data
    for v1, v2 in tqdm(paris):
        fp_sp[:, :, :, v1, :] = data[:, :, :, v1, :] - data[:, :, :, v2, :]

###############test##################
data = np.load('data/data117808/test_B_data.npy')
N, C, T, V, M = data.shape
fp_sp = open_memmap(
    'data/data117808/test_B_data_bone.npy',
    dtype='float32',
    mode='w+',
    shape=(N, 3, T, V, M))

fp_sp[:, :C, :, :, :] = data
for v1, v2 in tqdm(paris):
    fp_sp[:, :, :, v1, :] = data[:, :, :, v1, :] - data[:, :, :, v2, :]

100%|██████████| 24/24 [00:03<00:00,  6.68it/s]


In [ ]:
########### 说明 ##########
# 我们使用了模型融合的方法。共有3个框架，STGCN、AAGCN和MSG3D，每个框架下有四个模型，分别是
# Joint（用关节点数据的第一次训练）、Joint-2（第二次训练，重新训练，和第一次的参数配置一样）
# Bone（用骨骼数据的第一次训练）、Bone-2（第二次训练，重新训练，和第一次的参数配置一样）
# 因此总共有12个模型

# 由于一个框架下每次训练结束后，进行下一个模型的训练需要修改模型的名称，因此需要手动修改yaml文件，
# 因此这里一个框架下只能训练一个模型（一键实现不了所有模型的训练，需要修改模型名和数据集）
# 因此我们这里只写上训练STGCN-Joint（STGCN框架下关节数据的第一次训练）

In [11]:
#训练和验证 STGCN-Joint
!python3.7 main.py -c configs/recognition/stgcn/stgcn_fsd.yaml --validate

[11/23 20:15:36] DALI is not installed, you can improve performance if use DALI
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/yaml/constructor.py:126: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  if not isinstance(key, collections.Hashable):
[11/23 20:15:37] DATASET : 
[11/23 20:15:37]     batch_size : 32
[11/23 20:15:37]     num_workers : 4
[11/23 20:15:37]     test : 
[11/23 20:15:37]         file_path : /home/aistudio/data/data117808/test_B_data.npy
[11/23 20:15:37]         format : SkeletonDataset
[11/23 20:15:37]         test_mode : True
[11/23 20:15:37]     test_batch_size : 1
[11/23 20:15:37]     test_num_workers : 0
[11/23 20:15:37]     train : 
[11/23 20:15:37]         file_path : /home/aistudio/split_dataset/train_data.npy
[11/23 20:15:37]         format : SkeletonDataset
[11/23 20:15:37]         label_path : /home/aistudio/split_dataset/train_label.np

In [20]:
#测试模型 STGCN-Joint
#测试之后保存的是score 不是预测类别,结果文件在work/PaddleVideo/score
!python3.7 main.py --test -c configs/recognition/stgcn/stgcn_fsd.yaml -w output/STGCN-Joint/STGCN-Joint_best.pdparams

[11/23 20:41:57] DALI is not installed, you can improve performance if use DALI
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/yaml/constructor.py:126: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  if not isinstance(key, collections.Hashable):
[11/23 20:41:58] DATASET : 
[11/23 20:41:58]     batch_size : 32
[11/23 20:41:58]     num_workers : 4
[11/23 20:41:58]     test : 
[11/23 20:41:58]         file_path : /home/aistudio/data/data117808/test_B_data.npy
[11/23 20:41:58]         format : SkeletonDataset
[11/23 20:41:58]         test_mode : True
[11/23 20:41:58]     test_batch_size : 1
[11/23 20:41:58]     test_num_workers : 0
[11/23 20:41:58]     train : 
[11/23 20:41:58]         file_path : /home/aistudio/split_dataset/train_data.npy
[11/23 20:41:58]         format : SkeletonDataset
[11/23 20:41:58]         label_path : /home/aistudio/split_dataset/train_label.np

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 